In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import os

In [9]:
%pip install google-colaboratory
from google.colab import drive
drive.mount('/content/drive/')

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement google-colaboratory (from versions: none)
ERROR: No matching distribution found for google-colaboratory


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Set paths and parameters
data_dir = '/content/drive/MyDrive/Garment_Data'  # Path to your dataset
target_size = (224, 224)  # Resize images to this size
batch_size = 16  # Batch size

In [ ]:
# Automatically determine the number of classes based on subfolder names
class_names = sorted(os.listdir(data_dir))
num_classes = len(class_names)

In [ ]:
# Data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% of the data for validation
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50
)

In [ ]:
# Plot accuracy and loss curves
plt.figure(figsize=(12, 6))

# Accuracy curve
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy Curve')
plt.legend()

# Loss curve
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate and generate confusion matrix
# Get true labels and predictions
validation_generator.reset()
y_true = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())
y_pred = model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Ensure alignment between confusion matrix and class labels
unique_labels = np.unique(y_true)
filtered_class_labels = [class_labels[i] for i in unique_labels]

# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred_classes, labels=unique_labels)

# Plot confusion matrix
plt.figure(figsize=(12, 12))
cmd = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=filtered_class_labels)
cmd.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
plt.title('Confusion Matrix')
plt.show()

# Save the model
model.save("garment_classifier.h5")


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred_classes)

# Calculate precision, recall, F1-score for each class
precision, recall, f1, support = precision_recall_fscore_support(
    y_true, y_pred_classes, labels=unique_labels, average=None
)

# Calculate overall precision, recall, and F1-score
macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
    y_true, y_pred_classes, labels=unique_labels, average="macro"
)

weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(
    y_true, y_pred_classes, labels=unique_labels, average="weighted"
)

# Print metrics for each class
print("Class-wise Metrics:")
for idx, label in enumerate(filtered_class_labels):
    print(f"Class: {label}")
    print(f"  Precision: {precision[idx]:.2f}")
    print(f"  Recall: {recall[idx]:.2f}")
    print(f"  F1-Score: {f1[idx]:.2f}")

# Print overall metrics
print("\nOverall Metrics:")
print(f"  Accuracy: {accuracy:.2f}")
print(f"  Macro Precision: {macro_precision:.2f}")
print(f"  Macro Recall: {macro_recall:.2f}")
print(f"  Macro F1-Score: {macro_f1:.2f}")
print(f"  Weighted Precision: {weighted_precision:.2f}")
print(f"  Weighted Recall: {weighted_recall:.2f}")
print(f"  Weighted F1-Score: {weighted_f1:.2f}")
